In [3]:
import numpy as np
import pandas as pd
import pickle
import random

from pathlib import Path
from tqdm import tqdm

tqdm.pandas()

In [36]:
# category = 'Beauty'
# category = 'DVDs'
# category = 'Food & Drink'
# category = 'Internet'
category = 'Games'

# signal = 'like'
signal = 'write'
# signal = 'both'

summary = False

In [5]:
votes_file = '../csvs/votes_4core_split_60_20_20.tsv'
aspects_reviews_file = '../csvs/ciao_4core_gpt_aspects.csv'
summary_reviews_file = '../csvs/ciao_4core_summary.csv'

votes_df = pd.read_csv(votes_file, sep='\t')
if summary:
    reviews_df = pd.read_csv(summary_reviews_file)
else:
    reviews_df = pd.read_csv(aspects_reviews_file)

In [6]:
positive_vote = [3, 4, 5]

written_review = {}
liked_reviews = {}
umap = {}
rmap = {}
ridx2text = {}

for idx, row in tqdm(votes_df.iterrows(), total=len(votes_df)):
    review_id = row['review_id']
    voter_id = row['voter_id']
    vote = row['vote']
    categ = row['category']
    split = row['split']
    
    if vote in positive_vote:
        if voter_id not in liked_reviews:
            liked_reviews[voter_id] = {}
        if split not in liked_reviews[voter_id]:
            liked_reviews[voter_id][split] = []
        liked_reviews[voter_id][split].append((review_id, categ))
        
    if voter_id in umap:
        continue
        
    umap[voter_id] = len(umap)
        
for idx, row in tqdm(reviews_df.iterrows(), total=len(reviews_df)):
    review_id = row['review_id']
    user_id = row['user_id']
    if summary:
        text = row['summary']
    else:
        text = row['aspects']
    
    if user_id not in written_review:
        written_review[user_id] = []
    written_review[user_id].append(review_id)
    
    if review_id in rmap:
        continue

    rmap[review_id] = len(rmap)
    ridx2text[rmap[review_id]] = text

100%|██████████| 135880/135880 [00:04<00:00, 27226.35it/s]


In [7]:
# # liked reviews
# like_analysis = []
# for u in liked_reviews:
#     lens = []
#     for split in ['train', 'valid', 'test']:
#         lens.append(len(liked_reviews[u].get(split, [])))
#     like_analysis.append(lens)
    
# like_analysis = pd.DataFrame(like_analysis, columns=['train', 'valid', 'test'])
# like_analysis.describe()

# written reviews
# write_analysis = []
# for u in written_review:
#     write_analysis.append(len(written_review[u]))
    
# write_analysis = pd.DataFrame(write_analysis, columns=['train'])
# write_analysis.describe()

In [8]:
like_history = 30
write_history = 10

def load_history(voter_id, rmap, written_review, liked_reviews, split, history_type, category=None):
    history_write = [rmap[i] for i in written_review[voter_id]]
    if split in liked_reviews.get(voter_id, []):
        liked_reviews_user = liked_reviews[voter_id][split]
    else:
        liked_reviews_user = []
    if category:
        history_like = [rmap[liked_pair[0]] for liked_pair in liked_reviews_user if liked_pair[1] != category]
    else:
        history_like = [rmap[liked_pair[0]] for liked_pair in liked_reviews_user]

    if history_type == 'write':
        history = np.random.permutation(history_write)[:write_history]
    elif history_type == 'like':
        history = np.random.permutation(history_like)[:like_history]
    else:  # 'both'
        history = (list(np.random.permutation(history_write)[:write_history]) +
                   list(np.random.permutation(history_like)[:like_history]))

    history = list(history)
    return history

In [9]:
train_set = pd.read_csv('../csvs/train_set.csv', converters={'y_true': eval})
train_set = train_set[train_set['category'] == category]

valid_set = pd.read_csv('../csvs/valid_set.csv', converters={'y_true': eval})
valid_set = valid_set[valid_set['category'] == category]

test_set = pd.read_csv('../csvs/test_set.csv', converters={'y_true': eval})
test_set = test_set[test_set['category'] == category]

In [39]:
train_seq = {}
train_cand = {}

for idx, row in tqdm(train_set.iterrows(), total=len(train_set)):
    pid = row['product_id']
    uidx = umap[row['voter_id']]
    candidates = {rmap[rid]: vote for rid, vote in row['y_true'].items()}
    answers = [rid for rid, vote in candidates.items() if vote in [3, 4, 5]]
    user_history = load_history(row['voter_id'], rmap, written_review, liked_reviews, 'train', signal, category)
    
    if len(candidates) == 0:
        break
    
    if uidx not in train_seq:
        train_seq[uidx] = []
    if uidx not in train_cand:
        train_cand[uidx] = []
        
    for answer in answers:
        train_seq[uidx].append(user_history + [answer])
        negative_cands = [c for c, vote in candidates.items() if vote not in [3, 4, 5]]
        train_cand[uidx].append(negative_cands)

100%|██████████| 48170/48170 [00:57<00:00, 838.16it/s] 


In [40]:
# val_seq = {}
# val_cand = {}
# val_labels = []

# for idx, row in tqdm(valid_set.iterrows(), total=len(valid_set)):
#     pid = row['product_id']
#     uidx = umap[row['voter_id']]
#     candidates = {rmap[rid]: vote for rid, vote in row['y_true'].items()}
#     answers = [rid for rid, vote in candidates.items() if vote in [3, 4, 5]]
#     user_history = load_history(row['voter_id'], rmap, written_review, liked_reviews, 'valid', signal, category)
    
#     if uidx not in val_seq:
#         val_seq[uidx] = []
#     if uidx not in val_cand:
#         val_cand[uidx] = []
        
#     for answer in answers:
#         val_seq[uidx].append(user_history + [answer])
#         val_cand[uidx].append([c for c in candidates.keys()])
#         val_labels.append(list(candidates.values()))

In [41]:
val_seq = {}
val_cand = {}
val_labels = {}

for idx, row in tqdm(valid_set.iterrows(), total=len(valid_set)):
    pid = row['product_id']
    uidx = umap[row['voter_id']]
    
    candidates = {rmap[rid]: vote for rid, vote in row['y_true'].items()}
    
    # candidates_to_shuffle = list(candidates.items())
    # random.shuffle(candidates_to_shuffle)
    # candidates = dict(candidates_to_shuffle)
    
    user_history = load_history(row['voter_id'], rmap, written_review, liked_reviews, 'test', signal, category)
    
    if uidx not in val_seq:
        val_seq[uidx] = []
    if uidx not in val_cand:
        val_cand[uidx] = []
    if uidx not in val_labels:
        val_labels[uidx] = []
        
    max_value = max(candidates.values())
    answer = random.choice([rid for rid, vote in candidates.items() if vote == max_value])
    
    val_seq[uidx].append(user_history + [answer])
    val_cand[uidx].append([c for c in candidates.keys()])
    # val_labels.append(sorted(list(candidates.values()), reverse=True))
    val_labels[uidx].append(candidates)

100%|██████████| 15829/15829 [00:07<00:00, 2233.05it/s]


In [42]:
# test_seq = {}
# test_cand = {}
# test_labels = []

# for idx, row in tqdm(test_set.iterrows(), total=len(test_set)):
#     pid = row['product_id']
#     uidx = umap[row['voter_id']]
#     candidates = {rmap[rid]: vote for rid, vote in row['y_true'].items()}
#     answers = [rid for rid, vote in candidates.items() if vote in [3, 4, 5]]
#     user_history = load_history(row['voter_id'], rmap, written_review, liked_reviews, 'test', signal, category)
    
#     if uidx not in test_seq:
#         test_seq[uidx] = []
#     if uidx not in test_cand:
#         test_cand[uidx] = []
        
#     for answer in answers:
#         test_seq[uidx].append(user_history + [answer])
#         test_cand[uidx].append([c for c in candidates.keys()])
#         test_labels.append(list(candidates.values()))

In [43]:
test_seq = {}
test_cand = {}
test_labels = {}

for idx, row in tqdm(test_set.iterrows(), total=len(test_set)):
    pid = row['product_id']
    uidx = umap[row['voter_id']]
    candidates = {rmap[rid]: vote for rid, vote in row['y_true'].items()}
    user_history = load_history(row['voter_id'], rmap, written_review, liked_reviews, 'test', signal, category)
    
    if uidx not in test_seq:
        test_seq[uidx] = []
    if uidx not in test_cand:
        test_cand[uidx] = []
    if uidx not in test_labels:
        test_labels[uidx] = []
        
    max_value = max(candidates.values())
    answer = random.choice([rid for rid, vote in candidates.items() if vote == max_value])
    
    test_seq[uidx].append(user_history + [answer])
    test_cand[uidx].append([c for c in candidates.keys()])
    test_labels[uidx].append(candidates)

100%|██████████| 15792/15792 [00:07<00:00, 2228.50it/s]


In [44]:
dataset = {
    'train_seq': train_seq,
    'train_cand': train_cand,
    'val_seq': val_seq,
    'val_cand': val_cand,
    'val_labels': val_labels,
    'test_seq': test_seq,
    'test_cand': test_cand,
    'test_labels': test_labels,
    'meta': ridx2text,
    'umap': umap,
    'smap': rmap
}

In [45]:
if summary:
    dataset_path = Path(
        f'/sise/bshapira-group/lilachzi/models/LlamaRec/data/preprocessed/ciao_{category.replace(" & ", "_")}_{signal}_summary/dataset.pkl'
    )
else:
    dataset_path = Path(
        f'/sise/bshapira-group/lilachzi/models/LlamaRec/data/preprocessed/ciao_{category.replace(" & ", "_")}_{signal}/dataset.pkl'
    )
dataset_path

PosixPath('/sise/bshapira-group/lilachzi/models/LlamaRec/data/preprocessed/ciao_Games_write/dataset.pkl')

In [46]:
with dataset_path.open('wb') as f:
    pickle.dump(dataset, f)

In [80]:
# evaluation = {
#     'valid_labels': val_labels,
#     'test_labels': test_labels
# }

In [81]:
# if summary:
#     evaluation_path = Path(
#         f'/sise/bshapira-group/lilachzi/models/LlamaRec/data/preprocessed/ciao_{category.replace(" & ", "_")}_{signal}_summary/evaluation.pkl'
#     )
# else:
#     evaluation_path = Path(
#         f'/sise/bshapira-group/lilachzi/models/LlamaRec/data/preprocessed/ciao_{category.replace(" & ", "_")}_{signal}/evaluation.pkl'
#     )

In [82]:
# with evaluation_path.open('wb') as f:
#     pickle.dump(evaluation, f)

In [131]:
all_seqs = []
all_cands = []
all_labels = []
for u in val_seq.keys():
    for seq, cand, labels in zip(val_seq[u], val_cand[u], val_labels[u]):
        answer = seq[-1]
        np.random.shuffle(cand)
        all_labels.append([labels[i] for i in cand])
        
        if len(cand) <= 19 + 1:
            # self.rng.shuffle(cand)
            all_seqs += [seq]
            all_cands += [cand]
        else:
            cand.pop(cand.index(answer))
            for i in range(0, len(cand), 19):
                all_seqs += [seq]
                batch = cand[i:i+19] + [answer]
                # self.rng.shuffle(batch)
                all_cands += [batch]

In [133]:
len(all_seqs), len(all_cands), len(all_labels)

(22919, 22919, 15829)

In [135]:
i = 0
batch_size = 20
for labels in all_labels:
    # labels = torch.tensor(labels)
    batches = len([i for i in range(0, len(labels) - 1, batch_size - 1)])
    # scores = model_scores[i:i+batches]
    # if len(scores) > 1:
    #     scores = scores / torch.sum(scores, dim=1, keepdim=True)
    # answers = model_labels[i:i+batches]
    i += batches

In [136]:
i

22919

In [28]:
llm_negative_sample_size = 19
window_size = 10 
step = 3

all_seqs = []
all_cands = []
all_labels = []
for u in val_seq.keys():
    for seq, cand, labels in zip(val_seq[u], val_cand[u], val_labels[u]):
        np.random.shuffle(cand)
        all_labels.append([labels[i] for i in cand])

        if len(cand) <= llm_negative_sample_size + 1:
            all_seqs += [seq]
            all_cands += [cand]
        else:
            for i in range(0, len(cand)- window_size + 1, step):
                all_seqs += [seq]
                all_cands += [cand[i:i+window_size]]

In [29]:
len(all_seqs), len(all_cands), len(all_labels)

(59515, 59515, 15829)

In [34]:
i = 0
for labels in all_labels[:100]:
    if len(labels) <= llm_negative_sample_size + 1:
        i += 1
    else:
        num_windows = len(range(0, len(labels) - window_size + 1, step))
        i += num_windows

In [35]:
i

239